# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import os

pd.set_option('display.max_column', None)

In [2]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)
player_data['birthday'] = pd.to_datetime(player_data['birthday'])

match_data = pd.read_sql("SELECT * FROM Match;", db)
match_data['date'] = pd.to_datetime(match_data['date'] )

league_data = pd.read_sql("SELECT * FROM League;", db)
team_data = pd.read_sql("SELECT * FROM Team;", db)

attributes_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)
attributes_data['date'] = pd.to_datetime(attributes_data['date'] )

**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [4]:
players_180_190 = player_data[(player_data['height'] >= 180) & (player_data['height'] <=190)].shape[0]
players_180_190

5925

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [5]:
players_1980 = player_data[player_data.apply(lambda x: x["birthday"].year == 1980, axis=1)].shape[0]
players_1980

434

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [6]:
highest_players = (player_data.
                        sort_values(by =['weight', 'player_name'], ascending = [False, True])
                        .head(10)
                        .player_name
                        .tolist()
                    )
highest_players

['Kristof van Hout',
 'Tim Wiese',
 'Jeroen Verhoeven',
 'Ishmael Miller',
 'Cassio',
 'Christopher Samba',
 'Lars Unnerstall',
 'Marcus Hahnemann',
 'Abdoulaye Faye',
 'Enoch Showunmi']

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [7]:
player_data['year' ]= player_data.birthday.dt.year
inner_data = (player_data[player_data['year'].isin(range(1980, 1990+1))]
                .sort_values(by =['year'])
                .groupby(['year'])
                .agg({'player_api_id': 'nunique'})
                .rename(columns ={'player_api_id' : 'cnt'})
                .reset_index()
                .to_dict('records')
        )
years_born_players = [(i['year'], i['cnt'] ) for i in inner_data]
years_born_players

[(1980, 434),
 (1981, 475),
 (1982, 487),
 (1983, 541),
 (1984, 607),
 (1985, 593),
 (1986, 710),
 (1987, 722),
 (1988, 802),
 (1989, 723),
 (1990, 696)]

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [24]:
name = player_data['player_name'].str.split().str[0]
data_adriano = player_data[name.str.contains('Adriano')]

adriano_mean, adriano_std = data_adriano.height.mean(),\
                            data_adriano.height.std(),


**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [10]:
dow_with_min_players_born = (player_data.birthday.dt.day_name()
                                        .value_counts()
                                        .reset_index()
                                        .rename(columns ={'index': 'day_of_week'})
                                        .sort_values(by ='birthday')
                                        .reset_index(drop=True)
                                        .iloc[0, 0]
                            )
dow_with_min_players_born

'Sunday'

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [11]:
league_most_matches = (pd.merge(match_data, league_data, left_on='league_id', right_on='id')
                        .groupby('name')
                        .size()
                        .reset_index()
                        .rename(columns = {0: 'cnt'})
                        .sort_values(by = ['cnt', 'name'], ascending = [False, True])
                        .reset_index(drop = True)
                        .iloc[0, 0]
                    )
league_most_matches

'England Premier League'

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [12]:
needed_col = [f'{t}_player_{n}' for t in ('home','away') for n in range(1,12)]

id_ = (match_data[['id'] + needed_col]
            .melt(id_vars =['id'])
            .dropna()
            .groupby(['value'])
            .agg({'id': 'count'})
            .sort_values(by =['id'], ascending=False)
            .reset_index()
            .iloc[0, 0]
            .astype('int')
        )
max_matches_player = player_data.query('player_api_id == @id_').player_name.tolist()[0]
max_matches_player

'Steve Mandanda'

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [13]:
cols_to_drop = [
    'id', 'date', 'player_api_id', 
    'player_fifa_api_id', 'preferred_foot', 
    'attacking_work_rate', 'defensive_work_rate'
]

corr_matrix = (attributes_data
                        .drop(cols_to_drop, axis=1)
                        .corr(method='pearson')
                        .abs()
                )
top_correlated_features = (corr_matrix
                                .where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
                                .unstack()
                                .dropna()
                                .sort_values( ascending=False)
                                .head(5)
                                .index
                                .tolist()
                        )
top_correlated_features

[('gk_reflexes', 'gk_positioning'),
 ('gk_reflexes', 'gk_handling'),
 ('gk_positioning', 'gk_handling'),
 ('sliding_tackle', 'standing_tackle'),
 ('standing_tackle', 'marking')]

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [14]:
cols_to_drop2 = [
    'id', 
    'player_fifa_api_id', 'preferred_foot', 
    'attacking_work_rate', 'defensive_work_rate'
]

features_table = (pd.merge(attributes_data.drop(cols_to_drop2,  axis=1)
                           ,player_data[['player_api_id', 'player_name']], how='left', on='player_api_id')
                     .sort_values(by='date')
                     .groupby('player_api_id')
                     .last()
                    #  .drop('date', axis=1)
)
neyamar  = np.where(features_table['player_name'].str.contains('Neymar'))[0][0]
data_matrix = features_table.drop(['date', 'player_name'], axis=1).to_numpy()
features_table['dist'] = np.linalg.norm(data_matrix[neyamar] - data_matrix, axis=1)
neymar_similarities = (features_table.sort_values(by=['dist','player_name'], ascending=[True,True])
                              .query('dist > 0')
                              .nsmallest(5,['dist'])
                              .player_name
                              .tolist()
)
neymar_similarities

['Paulo Dybala',
 'Eden Hazard',
 'Giovani dos Santos',
 'Arjen Robben',
 'Lionel Messi']

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [15]:
bundesliga_id = league_data[league_data.name == 'Germany 1. Bundesliga'].id.values[0] 
borussia_id = team_data[team_data.team_long_name == 'Borussia Dortmund'].team_api_id.values[0] 

borussia_bundesliga_2008_2009_matches = match_data[(match_data.league_id == bundesliga_id) &
                                                   (match_data.season == '2008/2009') &
                                                   (match_data.home_team_api_id == borussia_id)].shape[0]
borussia_bundesliga_2008_2009_matches

17

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [16]:
away_data = match_data[(match_data.league_id == bundesliga_id) & (match_data.season == '2008/2009')].groupby('away_team_api_id').size()
home_data = match_data[(match_data.league_id == bundesliga_id) & (match_data.season == '2008/2009')].groupby('home_team_api_id').size()


team_most_matches_bundesliga_2008_2009 = (pd.concat([away_data.reset_index().rename(columns={'away_team_api_id' : 'id', 0: 'cnt'}),
                                                    home_data.reset_index().rename(columns={'home_team_api_id' : 'id', 0: 'cnt'})])
                                            .groupby('id')
                                            .agg({'cnt': 'sum'})
                                            .reset_index()
                                            .cnt
                                            .max()
                                        )
team_most_matches_bundesliga_2008_2009

34

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [17]:
arsenal_won_matches_2015_2016 = match_data[(match_data.season == '2015/2016') &
                                           (
                                                   (
                                                           (match_data.home_team_api_id == 9825) &
                                                           (match_data.home_team_goal > match_data.away_team_goal)
                                                   ) |
                                                   (
                                                           (match_data.away_team_api_id == 9825) &
                                                           (match_data.away_team_goal > match_data.home_team_goal)
                                                   )
                                           )].shape[0]
arsenal_won_matches_2015_2016

20

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [18]:
def get_winner_id(x):
    if (x['home_team_goal'] > x['away_team_goal']):
        return x['home_team_api_id']
    if (x['away_team_goal'] > x['home_team_goal']):
        return x['away_team_api_id']
    return np.nan

match_data['winner_id'] = match_data.apply(get_winner_id, axis=1)  
win_table = (match_data.query('season == "2015/2016"')
                            .groupby('winner_id')
                            .size()
                            .reset_index(name='cnt')
                            .astype(int)
                            # .sort_values(by ='cnt', ascending=False)
                 )
total_table = (match_data.query('season == "2015/2016"')
                    [['home_team_api_id','away_team_api_id']]
                    .stack()
                    .reset_index(name='team_api_id')
                    .groupby('team_api_id')
                    .size()
                    .astype(int)
                    .reset_index(name='total')
)
team_highest_winrate_2015_2016 = (pd.merge(total_table, win_table, left_on='team_api_id', right_on = 'winner_id')
                                    .assign(rate = lambda x: x['cnt'] / x['total'])
                                    .merge(team_data[['team_api_id','team_long_name']], how='left', on='team_api_id')
                                    .sort_values(by = ['rate', 'team_long_name'], ascending = [False, True])
                                    .nlargest(1,['rate'])
                                    .team_long_name
                                    .tolist()
                                    )[0]

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [19]:
team_id, league_id, season = 8658, 1729, '2010/2011'

highest_gap_england_2010_2011 = (match_data[(match_data['league_id'] == league_id) 
                                                & (match_data['season'] == season)
                                                & ((match_data['home_team_api_id'] == team_id) | (match_data['away_team_api_id'] == team_id))
                                            ]
                                                .sort_values(by='date')['date']
                                                .diff()
                                                .max()
                                                .days
                                )
highest_gap_england_2010_2011

18

### Warning! Do not change anything in the area below

In [20]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")